# This notebook exemplifies the calculation of the reduction to Pole by using the *classical equivalent layer* approach.

In [ ]:
import numpy
import scipy
from matplotlib import pyplot

In [ ]:
from codes.grids import regular_grid
from codes.equivalentlayer import layer, sensitivity_grav, sensitivity_mag

In [ ]:
n = 100
shape = (n, n)

In [ ]:
xo, yo, zo = regular_grid((-2000., 2000., -2000., 2000.), shape, -150.)

In [ ]:
layer = layer((-2000., 2000., -2000., 2000.), shape, 150.)

In [ ]:
inc, dec = 30., 50.
incs, decs = -45., -42.

In [ ]:
from codes.sphere import sphere_tfa
from codes.prism import prism_tf

In [ ]:
#tfa = sphere_tfa(xo, yo, zo, (0., 0., 200., 150., 1.), inc, dec, incs, decs)
tfp = prism_tf(xo, yo, zo, (-150., 150., -100., 200., 100., 550., 1.), inc, dec, incs, decs)

In [ ]:
from codes.auxiliars import addnoise
from codes.statistical import analysis

In [ ]:
#_ = analysis(tfa)
#print
_ = analysis(tfp)

In [ ]:
#tfa = addnoise(tfa, 2., 0.025)
tfp = addnoise(tfp, 3., 0.15)

In [ ]:
G = sensitivity_mag(xo, yo, zo, layer, inc, dec, incs, decs)

In [ ]:
T = numpy.abs(numpy.matrix.trace(G))
print 'Trace of matrix:', T

In [ ]:
I = numpy.identity(number**2)

In [ ]:
#solution = numpy.linalg.solve(numpy.dot(G.T,G) + (2.e-9)*I, numpy.dot(G.T,tfa))
solution = numpy.linalg.solve(numpy.dot(G.T, G) + (2.e-8)*I, numpy.dot(G.T, tfp))

In [ ]:
tf_pred = numpy.dot(G, solution)

In [ ]:
from codes.auxiliars import residual

In [ ]:
res, mean, deviation = residual(tfp, tf_pred)

In [ ]:
pyplot.figure(figsize=(16, 4))

pyplot.subplot(1, 3, 1)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), tfp.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('Dado observado (nT)')
pyplot.colorbar()

pyplot.subplot(1, 3, 2)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), tf_pred.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('Dado predito (nT)')
pyplot.colorbar()

pyplot.subplot(1, 3, 3)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), res.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('Residuo (nT)')
pyplot.colorbar()

pyplot.show()

### Applying the reduction to Pole by using Equivalent Layer

In [ ]:
inc_polo = 90.
dec_polo = 0.

In [ ]:
# Calculo da RTP verdadeira
#true_rtp = sphere_tfa(xo, yo, zo, (0., 0., 200., 150., 1.), inc_polo, dec_polo)
true_rtp = prism_tf(xo, yo, zo, (-150., 150., -100., 200., 100., 550., 1.), inc_polo, dec_polo)

In [ ]:
# Calculo da RTP via Fourier
from codes.filtering import reduction

In [ ]:
rtp = reduction(xo.reshape(n,n), yo.reshape(n,n), tfp.reshape(n,n), incs, decs, inc, dec)

In [ ]:
# Calculo da RTP via camada equivalente
G_rtp = sensitivity_mag(xo, yo, zo, layer, inc_polo, dec_polo, inc_polo, dec_polo)

In [ ]:
tf_rtp = numpy.dot(G_rtp, solution)

In [ ]:
pyplot.figure(figsize=(16, 4))

pyplot.subplot(1, 3, 1)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), true_rtp.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('RTP verdadeira (nT)')
pyplot.colorbar()

pyplot.subplot(1, 3, 2)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), rtp.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('RTP via Fourier (nT)')
pyplot.colorbar()

pyplot.subplot(1, 3, 3)
pyplot.contourf(yo.reshape(n,n), xo.reshape(n,n), tf_rtp.reshape(n,n), 20, cmap = pyplot.cm.jet)
pyplot.title('RTP via camada (nT)')
pyplot.colorbar()

pyplot.show()